# Decision Tree Classifier

### Source data input

In [1]:
import h5py
import numpy as np
import os
import scipy as sp
from math import sqrt
import pandas



with h5py.File('./Input/images_training.h5','r') as H:
    data_train = np.copy(H['datatrain'])
with h5py.File('./Input/labels_training.h5','r') as H:
    label_train = np.copy(H['labeltrain'])
    
with h5py.File('./Input/images_testing.h5','r') as H:
    data_test = np.copy(H['datatest'])
with h5py.File('./Input/labels_testing_2000.h5','r') as H:
    label_test = np.copy(H['labeltest'])

print(data_train.shape,label_train.shape)
print(data_test.shape)

OSError: Unable to open file (unable to open file: name = './Input/images_training.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### Check Source Data

In [ ]:
import matplotlib.pyplot as plt
data_train = data_train.reshape((data_train.shape[0], 28, 28))
data_test=data_test.reshape((data_test.shape[0], 28, 28))
plt.imshow(data_train[0], cmap=plt.get_cmap('gray'))
plt.title("class " + str(label_train[0]) + ": T-shirt/Top" )
plt.show()

### Date Pre-processing

As the values are continuous float, we need to bin the data, i.e. put the values into categories based on their value. We did this by multiplying all values by 10 and converting them into integers and as such we created 10 categories/bins labelled as 0 to 9. 

In [18]:
#convert the array value into int in order to bin the data
def Cover_to_int(array_x):
	x=np.multiply(array_x, 10)
	return x.astype(int)

#create val set
data_val = Cover_to_int(data_train[25000:30000,])
partial_train = Cover_to_int(data_train[:25000,])

#create val labels
label_val = label_train[25000:30000,].astype('int')
partial_label_train = label_train[:25000,].astype('int')


#create testing set
data_test=Cover_to_int(data_test[:2000])

print(data_val.shape)

(50, 28, 28)


For data transformation,we use numpy.reduceat function to combine every 4 adjacent values on the matrix into 1 value by taking the their average

In [19]:
#reshape data
def reshape_matrix(x):
    x_1=np.add.reduceat(x,np.arange(0,28,2),1)
    x_2=np.add.reduceat(x_1,np.arange(0,28,2),2)
    x_3=x_2.reshape(x_2.shape[0],-1)
    x_v=np.multiply(x_3, 0.25).astype(int)
    return x_v

flat_partial_train = reshape_matrix(partial_train)
flat_data_val = reshape_matrix(data_val)
flat_data_test = reshape_matrix(data_test)

Check the date distribution between bins 

In [20]:
uniqueValues, Count = np.unique(flat_partial_train, return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", Count)

Unique Values :  [ 0  1  2  3  4  5  6  7  8  9 10]
Occurrence Count :  [23721  2250  2313  2517  3009  3054  3201  3634  4361   939     1]


As the count is classs 9 and 10 are significantly lower than other bins, they are combined into bin 9

In [21]:
flat_partial_train[flat_partial_train > 8] = 9

Combine training data with lebals

In [23]:
#covert the matrix into 2 dimension
train_label_reshaped=partial_label_train.reshape(partial_label_train.shape[0],1)

#add the label data as the last feature to the train data
Train_data = np.hstack((flat_partial_train,train_label_reshaped))

#covert the matrix into 2 dimension
test_label_reshaped=label_val.reshape(label_val.shape[0],1)

#add the label data as the last feature to the train data
Test_data = np.hstack((flat_data_val,test_label_reshaped))

#covert the matrix into 2 dimension
access_label_reshaped=label_test.reshape(label_test.shape[0],1)

#add the label data as the last feature to the train data
Access_data = np.hstack((flat_data_test,access_label_reshaped))


### Decision Tree

In [24]:
# Split dataset according the the feature (ref) and criterion (value)
def data_split(ref, value, dataset):
	l, r = list(), list()   
    #check all rows in dataset
	for row in dataset:
        #if a value in the dataset is less then the criterion, put it to the left node, otherwise put it to the right node
		if row[ref] < value:
			l.append(row)
		else:
			r.append(row)
	return l, r
 
# Calculate the Gini index
def gini_index(DataSplit, classes):
	# count all samples at split point
	count_samples = float(sum([len(group) for group in DataSplit]))
	# sum weighted Gini ref for each group
	gini = 0.0
	for group in DataSplit:
        # calculate number of samples in the group
		size = float(len(group))
        # set initial score at 0
		score = 0.0
		# avoid divide by zero by giving it 0 score directly
		if size == 0:
			score = 0.0
		else:			        
            # score the group based on the score for each class
			for class_value in classes:
				class_sub=  [row[-1] for row in group]
                # calculate Proportion = number of classes / number of samples in the group
				p = class_sub.count(class_value) / size
				score += p **2
		# Gini index = [1-∑ (proportion * proportion)] * (group size/ dataset size)
		gini += (1.0 - score) * (size / count_samples)
	return gini
 
# Select the best split point for a dataset
def get_split(dataset):
	unique_class_val = list(set(row[-1] for row in dataset))
	b_score=9999
    # get the number of columns  
	n_feature = len(dataset[0])-1 
	for ref in range(n_feature):
        # get unique values in the column  
		unique_value=set([row[ref] for row in dataset])
		for criteria in unique_value:
            #skip the class 0 and 9 as they cannot split the dataset into two groups
			if criteria==0 or criteria==9:
				continue       
			DataSplit = data_split(ref, criteria, dataset)
			gini = gini_index(DataSplit, unique_class_val)
			if gini < b_score:
				b_ref = ref 
				b_value = criteria 
				b_score = gini 
				b_DataSplit = DataSplit
	return {'ref':b_ref, 'value':b_value, 'branch':b_DataSplit}
 
# Create a terminal node value
def terminal(group):
	result_class_val = [row[-1] for row in group]
    #use most common class value as the output for terminal node
	output=max(set(result_class_val), key=result_class_val.count)
	return output
 
# Create child-node and terminal node
def split(node, max_depth, min_leaf_size, depth):
	l, r = node['branch']
    #delete node as it is not needed
	del(node['branch'])
	len_l=len(l)
	len_r=len(r)	
	# if a split is a pure split, i.e. all value in a split belong to the same group. 
    # we wouldn’t be able to further split the data.
	if not l or not r:
		node['l'] = node['r'] = terminal(l + r)
		return
	# if it is over the max_depth, no further split required
	if depth >= max_depth:
		node['l'], node['r'] = terminal(l), terminal(r)
		return
	# if leave size less than min_leaf_size, no further split required
	if len_l <= min_leaf_size:      
		node['l'] = terminal(l)
	else:
        # if leave size is bigger than min_leaf_size, proceed with further split
		node['l'] = get_split(l)
		depth+=1
		split(node['l'], max_depth, min_leaf_size, depth)
	# if leave size less than min_leaf_size, no further split required
	if len_r <= min_leaf_size:
		node['r'] = terminal(r)
	else:
        # if leave size is bigger than min_leaf_size, proceed with further split
		node['r'] = get_split(r)
		depth+=1
		split(node['r'], max_depth, min_leaf_size, depth)
 
 # Call all functions needed to build the decision tree
def build_tree(train, max_depth, min_leaf_size):
	tree = get_split(train)
    # set initial depth as 1 as the split starting from root
	split(tree, max_depth, min_leaf_size, 1)
	return tree
 
# Predict with decision tree
def predict(tree, row):
    # tree has three components: ref, value, branch
    # the below code compares data value to criteria for split and determine if the left or right branch should be followed
	if row[tree['ref']] < tree['value']:
        # check if the a ternimal node is reached
		if isinstance(tree['l'], dict):
			return predict(tree['l'], row)
		else:
			return tree['l']
	else:
        # check if the a ternimal node is reached
		if isinstance(tree['r'], dict):
			return predict(tree['r'], row)
		else:
			return tree['r']

 
 # combin everything together
def decision_tree(train, test, max_depth, min_leaf_size):
	tree = build_tree(train, max_depth, min_leaf_size)
    # set prediction result as list
	predictions = list()
    # preform the prediction row by row
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)


### Predict with decision tree

In [25]:
pred = decision_tree(Train_data,Access_data,19,15)

### Access prediction accuracy

In [28]:
def accuracy(pred,y):
    Correct_count=0
    for i in range(0,len(pred)):
        if pred[i]==y[i]:
            Correct_count+=1 
    return Correct_count/len(y)

Pred_Accuracy = accuracy(pred,label_test)
print(Pred_Accuracy)

0.6085
